<a href="https://colab.research.google.com/github/josemoragonzalez/ejemplosColab/blob/master/ApiChC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tercer Ejercicio, Api desarrolladores ChC

In [ ]:
import pandas as pd
import requests

In [ ]:
# Agregar tu ticket
ticket = 'C495FBFD-6480-46C6-8A1A-EAE762EC0271'

In [ ]:
def compradores(ticket):
  url = 'http://api.mercadopublico.cl/servicios/v1/Publico/Empresas/BuscarComprador?ticket='+ticket
  r = requests.get(url)
  comp_json = r.json()
  df = pd.json_normalize(comp_json['listaEmpresas']) 
  return df

In [ ]:
dfcomp = compradores(ticket)
dfcomp

,CodigoEmpresa,NombreEmpresa
0,1034640,CORPORACION MUNICIPAL DE PEÑALOLEN PARA EL DE...
1,7311,Hospital de enfermedades InfecciDr. Lucio Cór...
2,7086,AGCID
3,1224636,Agencia Chilena de Eficiencia Energética
4,7193,Agencia de Promoción de la Inversión Extranjera
...,...,...
856,96996,Universidad de Valparaíso
857,99805,Universidad del Bío-Bío
858,164268,UNIVERSIDAD METROPOLITANA DE CIENCIAS DE LA ED...
859,126109,Universidad Tecnológica Metropolitana


In [ ]:
def ocxfecha(fecha,ticket):
  url = 'http://api.mercadopublico.cl/servicios/v1/publico/ordenesdecompra.json?fecha='+fecha+'&ticket='+ticket
  r = requests.get(url)
  comp_json = r.json()
  df = pd.json_normalize(comp_json['Listado']) 
  return df['Codigo']

In [ ]:
dfocs = ocxfecha('19072020', ticket)
dfocs

0           1026-40-SE20
1       1040758-245-CM20
2      1057417-5451-SE20
3      1057417-5828-SE20
4       1057420-460-SE20
             ...        
573       947621-63-SE20
574       947621-64-CM20
575       947621-72-CM20
576       947621-74-AG20
577         993-287-SE20
Name: Codigo, Length: 578, dtype: object

In [ ]:
def ocd(codigo, ticket):
  import time
  time.sleep(2)
  url = 'http://api.mercadopublico.cl/servicios/v1/publico/ordenesdecompra.json?codigo='+codigo+'&ticket='+ticket
  r = requests.get(url)
  comp_json = r.json()
  df = pd.json_normalize(comp_json['Listado']) 
  return df

In [ ]:
dftest = ocd('1057501-6535-SE20', ticket)
dftest

,Codigo,Nombre,CodigoEstado,Estado,CodigoLicitacion,Descripcion,CodigoTipo,Tipo,TipoMoneda,CodigoEstadoProveedor,EstadoProveedor,TieneItems,PromedioCalificacion,CantidadEvaluacion,Descuentos,Cargos,TotalNeto,PorcentajeIva,Impuestos,Total,Financiamiento,Pais,TipoDespacho,FormaPago,Fechas.FechaCreacion,Fechas.FechaEnvio,Fechas.FechaAceptacion,Fechas.FechaCancelacion,Fechas.FechaUltimaModificacion,Comprador.CodigoOrganismo,Comprador.NombreOrganismo,Comprador.RutUnidad,Comprador.CodigoUnidad,Comprador.NombreUnidad,Comprador.Actividad,Comprador.DireccionUnidad,Comprador.ComunaUnidad,Comprador.RegionUnidad,Comprador.Pais,Comprador.NombreContacto,Comprador.CargoContacto,Comprador.FonoContacto,Comprador.MailContacto,Proveedor.Codigo,Proveedor.Nombre,Proveedor.Actividad,Proveedor.CodigoSucursal,Proveedor.NombreSucursal,Proveedor.RutSucursal,Proveedor.Direccion,Proveedor.Comuna,Proveedor.Region,Proveedor.Pais,Proveedor.NombreContacto,Proveedor.CargoContacto,Proveedor.FonoContacto,Proveedor.MailContacto,Items.Cantidad,Items.Listado
0,1057501-6535-SE20,CONVENIOS Marcapasos - Abril 2020,6,Aceptada,1057501-181-LR19,1. Convenio desde Lic. Pública 1057501-181-LR1...,8,SE,CLP,4,Aceptada,1,0.0,0,0.0,0.0,10017000.0,0.0,0.0,10017000.0,,CL,7,2,2020-07-14T12:50:50.48,2020-07-15T17:57:17.413,2020-07-19T22:04:25.753,None,2020-07-15T17:21:00,7320,Complejo Asistencial Dr. Sotero del Rio,61.608.502-6,1057501,Bienes y Servicios,Servicio Público,"Avenida Melchor Concha y Toro 3459, Puente Alto",Puente Alto,Región Metropolitana de Santiago,CL,Carlos Vargas,,56-2-25766046,cvargasa@ssmso.cl,1342684,CICARSOT S. A.,OTRAS ACTIVIDADES EMPRESARIALES RELACIONADAS C...,743873,CICARSOT S. A.,76.241.497-k,Portugal 20 of.27,Santiago Centro,Región Metropolitana de Santiago,CL,Demian Andrés Fullerton Moreno,Gerente General,56-9-98248245,dfullert@gmail.com,3,"[{'Correlativo': 1, 'CodigoCategoria': 8512160..."


In [ ]:
pd.json_normalize(dftest['Items.Listado'].to_dict()[0])

,Correlativo,CodigoCategoria,Categoria,CodigoProducto,Producto,EspecificacionComprador,EspecificacionProveedor,Cantidad,Unidad,Moneda,PrecioNeto,TotalCargos,TotalDescuentos,TotalImpuestos,Total
0,1,42281600,Equipamiento y suministros médicos / Productos...,42281602,Soluciones de glutaraldehído,ALCOHOL GEL CON ALOE VERA EN ENVASE DE 1 LITRO,ALCOHOL GEL CON ALOE VERA EN ENVASE DE 1 LITRO,97.0,None,CLP,6900.0,0.0,0.0,0.0,669300.0
1,2,53131600,"Ropa, maletas y productos de aseo personal / A...",53131606,Desodorantes,DESINFECTANTE DE AMBIENTES EN AEROSOL SIMILAR ...,DESINFECTANTE DE AMBIENTES EN AEROSOL SIMILAR ...,100.0,None,CLP,2690.0,0.0,0.0,0.0,269000.0


In [ ]:
def oc_fecha_con_detalle(fecha, ticket):
  listoc = ocxfecha(fecha,ticket)
  dfoctotal = pd.DataFrame()
  for oc in listoc:
    dfoctotal.append(ocd(oc, ticket))
  
  return dfoctotal

In [ ]:
import time

dfprueba =  oc_fecha_con_detalle('19072020', ticket)
dfprueba

""


In [ ]:
dfprueba

""


In [ ]:
dfoctotal.append(ocd(oc, ticket))
  
  return dfoctotal

In [ ]:
dfprueba


NameError: ignored